# Tutorial: Climate Networks

The objective of this tutorial is to introduce climate networks and explain and illustrate their application with the pyunicorn package. First some theoretical background for understanding general climate networks will be given and then some methods provided by `pyunicorn.climate.ClimateNetwork` will be illustrated. An introduction and application of coupled climate networks will follow. For a detailed discussion and further references, please consult __[Donges et al., 2015](https://aip.scitation.org/doi/10.1063/1.4934554)__, on which this tutorial is based. 